## Amaç 
Akciğer görüntüleri verilen  veri setini YoloV5 modeli kullanılarak transfer öğrenme yardımıyla virüsün hangi bölgede olduğunun işaretlenmesi yapay zeka ile yapıalcaktır.


## 🖼️ YoloV5 NEDİR ?
YoloV5 algoritması nesneleri kutucuk içerisine alarak koordinatlarını ve sınfını döndüren bir nesne tespit algoritmasıdır. Coco dataseti ile eğitilmiştir. Doğruluğunun ve performansının yüksek olması sebebiyle tercih ettim.
 

Aşamalar:

✔️ Gerekli dosya yapısını oluştur <br>
✔️ Bounding box (kutucukları) yolov5 için uygun formata getir   <br>
✔️ Hafif bir YoloV5 modeliyle eğitim yap <br>
✔️ Sonnuçları incele <br>
✔️ Test görüntülerine bak ve  görselleştir <br>

# ☀️ İmportlar ve kurulum

YoloV5 dosya yapısına göre mimari aşağıdaki gibi olmalıdır. 

```
/parent_folder
    /dataset
         /images
         /labels
    /yolov5
```

* /temp dizini oluşturacağız <br>
* YoloV5 indirilip gerekli bağımlılıklar kurulacak <br>

In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login()

In [ ]:
%cd /kaggle
! mkdir tmp
%cd tmp

In [ ]:
#  YOLOv5 modelini indir
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
# Bağımlılıkların yüklenmesi
%pip install -qr requirements.txt 

%cd ../
import torch
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

In [ ]:
# kütüphanlerin içe aktarılması
import os
import gc
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from shutil import copyfile
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

### Sabitlerin tanımlanması

In [ ]:
# eğitim görüntü yolu
TRAIN_PATH = 'input/siim-covid19-resized-to-256px-jpg/train/'
# resimlerin gelmesi gerekn boyut
IMG_SIZE = 256
# her seferinde çekilecek sayısı
BATCH_SIZE = 16
# kaç sefer veriler verilecek
EPOCHS = 35

## 🔨 Veri Setinin Hazırlanması ¶ 
YoloV5 nesne tepsiti için önemli bölümdür. Dizin yapısı ve bounding box (kare kutu) YoloV5 modelini eğitmek için uygun formatta olmalıdır

Ben veri setini  yeniden boyutlandırarak kullanıyorum. Resimlerin boyutunu 256x256 formatına getiriyorum.

<ol>
    <li> train-validation ( eğitim-doğrulama ) bölmesi </li> 
        <li> Gerekli /dataset klasör yapısını ve klasörlerdeki görüntüleri yarat </li>  
        <li> Gerekli /dataset klasör yapısını ve klasörlerdeki görüntüleri yarat </li>  
        <li> YoloV5 modelini eğitmek için data.yaml dosyasını oluştur </li>  
            <li> YoloV5 formatı için  bounding box (kare kutucukları) yarat </li>  


</ol>

In [ ]:
# Herşey /kaggle dizininden
%cd ../
# resimlerin csv formattaki dosyasını al
df = pd.read_csv('input/siim-covid19-detection/train_image_level.csv')

# id sütunundaki değerleri değiştir
df['id'] = df.apply(lambda row: row.id.split('_')[0], axis=1)
# Kesin yolu ekle
df['path'] = df.apply(lambda row: TRAIN_PATH+row.id+'.jpg', axis=1)
# Resimlerin image_level değerlerini al
df['image_level'] = df.apply(lambda row: row.label.split(' ')[0], axis=1)

df.head(5)

In [ ]:
# meta.csv dosyasını yükle
# Orjinal boyutların bounding box formatı için yeniden scale (ölçeklendirilmesi) gerekmektedir
meta_df = pd.read_csv('input/siim-covid19-resized-to-256px-jpg/meta.csv')
train_meta_df = meta_df.loc[meta_df.split == 'train']
train_meta_df = train_meta_df.drop('split', axis=1)
train_meta_df.columns = ['id', 'dim0', 'dim1']

train_meta_df.head(2)

In [ ]:
# Her iki dataframe'i  birleştirme
df = df.merge(train_meta_df, on='id',how="left")
df.head(2)

### 🍘 Eğitim-doğrulama bölmesi

In [ ]:
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df.image_level.values)

train_df.loc[:, 'split'] = 'train'
valid_df.loc[:, 'split'] = 'valid'

df = pd.concat([train_df, valid_df]).reset_index(drop=True)

In [ ]:
print(f'Veri seti boyutu: {len(df)}, Eğitim görüntüleri: {len(train_df)}. Doğrulama görüntüleri: {len(valid_df)}')

## 🍚 Prepare Required Folder Structure
Model için gerekli olan veri seti dosya yapısı aşağıdaki gibidir:
<br>
``` 
/parent_folder
    /dataset
         /images
             /train
             /val
         /labels
             /train
             /val
    /yolov5
```

Yolu covid olarak isimnlendirdim

In [ ]:
os.makedirs('tmp/covid/images/train', exist_ok=True)
os.makedirs('tmp/covid/images/valid', exist_ok=True)

os.makedirs('tmp/covid/labels/train', exist_ok=True)
os.makedirs('tmp/covid/labels/valid', exist_ok=True)

! ls tmp/covid/images

Eğitim ve doğrulama görüntülerini tutacağım klasörleri oluşturdum (train-valid). <br>
Aynı zamanda etiklleri ve bounding box bbilgilerini tutmak için ayrı bir olark klasör oluşturdum

In [ ]:
for i in tqdm(range(len(df))):
    row = df.loc[i]
    if row.split == 'train':
        copyfile(row.path, f'tmp/covid/images/train/{row.id}.jpg')
    else:
        copyfile(row.path, f'tmp/covid/images/valid/{row.id}.jpg')

Görüntüleri kopyalayıp yeni dizinlere yapıştırdım

# Create .YAML file
data.yaml,  veri seti yapılandırma dosyasıdır.
1. otomatik indirme için bir "isteğe bağlı" indirme komutu/URL'si
2. eğitim görüntüleri klasörünün yolu (veya eğitim görüntülerinin listesini içeren bir *.txt dosyasının yolu)
3. doğrulama görüntüleri dizininin yolu (veya doğrulama görüntülerinin listesini içeren bir *.txt dosyasının yolu)
4. sınıf sayısı
5. sınıf isimleri listesi

In [ ]:
#.yaml dosyası oluştur
import yaml

data_yaml = dict(
    train = '../covid/images/train',
    val = '../covid/images/valid',
    nc = 2,
    names = ['none', 'opacity']
)

# Dosyayı yolov5/data/ klasöründe  oluşturma
with open('tmp/yolov5/data/data.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)
    
%cat tmp/yolov5/data/data.yaml

> **YOLOv5 için Koordineli Sınırlama Kutusu Hazırlayın

Sınırlayıcı kutuya/kutulara sahip her görüntü için, aşağıda gösterilen biçimde görüntüyle aynı ada sahip bir .txt dosyası oluşturulacaktır:
1.  Nesne başına bir satır
2. Her satır, sınıf x_center y_center genişlik yükseklik biçimidir.
3. Kutu koordinatları normalleştirilmiş xywh formatında olmalıdır (0 - 1 arası). x_center ve genişliği görüntü genişliğine ve y_center ve yüksekliği görüntü yüksekliğine bölerek piksel cinsinden kutularla normalleştirebiliriz.
4. Class numbers are zero-indexed (start from 0).

In [ ]:
#Etiket sütununun satır değerini ayrıştırarak ham sınırlayıcı kutuyu alın.
def get_bbox(row):
    bboxes = []
    bbox = []
    for i, l in enumerate(row.label.split(' ')):
        if (i % 6 == 0) | (i % 6 == 1):
            continue
        bbox.append(float(l))
        if i % 6 == 5:
            bboxes.append(bbox)
            bbox = []  
            
    return bboxes
        
     

In [ ]:
# Yeniden boyutlandırılan görüntünün boyutuna göre sınırlayıcı kutuları ölçekleyin.
def scale_bbox(row, bboxes):
    # Ölçeklendirme faktörünü al
    scale_x = IMG_SIZE/row.dim1
    scale_y = IMG_SIZE/row.dim0
    
    scaled_bboxes = []
    for bbox in bboxes:
        x = int(np.round(bbox[0]*scale_x, 4))
        y = int(np.round(bbox[1]*scale_y, 4))
        x1 = int(np.round(bbox[2]*(scale_x), 4))
        y1= int(np.round(bbox[3]*scale_y, 4))

        scaled_bboxes.append([x, y, x1, y1]) # xmin, ymin, xmax, ymax
        
    return scaled_bboxes

In [ ]:
# Sınırlayıcı kutuları YOLO formatında dönüştürün.
def get_yolo_format_bbox(img_w, img_h, bboxes):
    yolo_boxes = []
    for bbox in bboxes:
        w = bbox[2] - bbox[0] # xmax - xmin
        h = bbox[3] - bbox[1] # ymax - ymin
        xc = bbox[0] + int(np.round(w/2)) # xmin + width/2
        yc = bbox[1] + int(np.round(h/2)) # ymin + height/2
        
        yolo_boxes.append([xc/img_w, yc/img_h, w/img_w, h/img_h]) # x_center y_center genişlik yüksekliği
    
    return yolo_boxes

In [ ]:
# Sınırlayıcı kutu için txt dosyalarını hazırlayın
for i in tqdm(range(len(df))):
    row = df.loc[i]
    # resim kimliğini al
    img_id = row.id
    # Bölünmek
    split = row.split
    # Görüntü düzeyinde etiket alın
    label = row.image_level
    
    if row.split=='train':
        file_name = f'tmp/covid/labels/train/{row.id}.txt'
    else:
        file_name = f'tmp/covid/labels/valid/{row.id}.txt'
        
    
    if label=='opacity':
         # kutuları al
        bboxes = get_bbox(row)
        # Sınırlayıcı kutuları ölçeklendir
        scale_bboxes = scale_bbox(row, bboxes)
        # YOLOv5 formatı
        yolo_bboxes = get_yolo_format_bbox(IMG_SIZE, IMG_SIZE, scale_bboxes)
        
        with open(file_name, 'w') as f:
            for bbox in yolo_bboxes:
                bbox = [1]+bbox
                bbox = [str(i) for i in bbox]
                bbox = ' '.join(bbox)
                f.write(bbox)
                f.write('\n')

# Eğitim

In [ ]:
%cd tmp/yolov5/

In [ ]:
!python train.py --img {IMG_SIZE} \
                 --batch {BATCH_SIZE} \
                 --epochs {EPOCHS} \
                 --data data.yaml \
                 --weights yolov5s.pt \
                --freeze 13

## Test

In [ ]:
TEST_PATH = '/kaggle/input/siim-covid19-resized-to-256px-jpg/test/' # test resimlerinin olduğu yer

In [ ]:
MODEL_PATH = './runs/train/exp/weights/best.pt' #modelin olduğu yer

In [ ]:
!python detect.py --weights {MODEL_PATH} \
                  --source {TEST_PATH} \
                  --img {IMG_SIZE} \
                  --conf 0.281 \
                  --iou-thres 0.5 \
                  --max-det 3 \
                  --save-txt \
                  --save-conf

In [ ]:
import glob
#birkaç resmi diziye alma
paths = []
for i in glob.glob("./runs/detect/exp2/"+"*.jpg")[200:250]:
    paths.append(i)

In [ ]:
paths

## Test sonuçları

In [ ]:
plt.figure(figsize=(16,10))
for i in range(49):
    plt.subplot(7,7,(i+1))
    plt.imshow(cv2.imread(paths[i]))